In [24]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import (ElasticNet, Lasso, LinearRegression, Ridge,
                                  TheilSenRegressor)
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from joblib import dump

### Getting the data from the transformed data set

In [2]:
df = pd.read_csv('data/transformed_data.csv')

In [3]:
df_copy = df.copy()
display(df_copy)

,gender,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,Residence_type_Rural,Residence_type_Urban
0,0,67.0,0,1,1,228.69,36.600000,1,0,0,1,0,0,1,0,0,0,1
1,1,61.0,0,0,1,202.21,28.893237,1,0,0,0,1,0,0,1,0,1,0
2,0,80.0,0,1,1,105.92,32.500000,1,0,0,1,0,0,0,1,0,1,0
3,1,49.0,0,0,1,171.23,34.400000,1,0,0,1,0,0,0,0,1,0,1
4,1,79.0,1,0,1,174.12,24.000000,1,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,1,80.0,1,0,1,83.75,28.893237,0,0,0,1,0,0,0,1,0,0,1
5106,1,81.0,0,0,1,125.20,40.000000,0,0,0,0,1,0,0,1,0,0,1
5107,1,35.0,0,0,1,82.99,30.600000,0,0,0,0,1,0,0,1,0,1,0
5108,0,51.0,0,0,1,166.29,25.600000,0,0,0,1,0,0,1,0,0,1,0


In [4]:
y = df_copy['stroke'].to_numpy()
del df_copy['stroke']
x = df_copy.to_numpy()

### Splitting the dataset with 0.7 train size

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

### Training the model

In [6]:
lir = LinearRegression().fit(X_train, y_train)
rfr = RandomForestRegressor().fit(X_train, y_train)
gbr = GradientBoostingRegressor().fit(X_train, y_train)
dtr = DecisionTreeRegressor().fit(X_train, y_train)
knr = KNeighborsRegressor().fit(X_train, y_train)
ridge = Ridge().fit(X_train, y_train)
lasso = Lasso().fit(X_train, y_train)
en = ElasticNet().fit(X_train, y_train)
tsr = TheilSenRegressor().fit(X_train, y_train)

In [7]:
models = [
  ['Linear Regression', lir],
  ['Random Forest', rfr],
  ['Gradient Boosting', gbr],
  ['Decision Tree', dtr],
  ['K Neighbors', knr],
  ['Ridge', ridge],
  ['Lasso', lasso],
  ['Elastic Net', en],
  ['TheilSen', tsr],
]

The function below is to calculate the mean squared error and mean absolute error of predicted values

In [8]:
def evaluate(model, X, y):
  y_pred = model.predict(X)
  mse = mean_squared_error(y, y_pred)
  mae = mean_absolute_error(y, y_pred)
  return mse, mae

The function below is to use the function above and compare the results and return the best model and its data alongside the mse and mae of all the models

In [9]:
def compare_models(models, X, y):
  best_mse = 0
  best_mae = 0
  best_model = None
  results = pd.DataFrame(columns=['mse', 'mae'])
  for i in range(len(models)):
    mse, mae = evaluate(models[i][1], X, y)
    results.loc[models[i][0]] = [mse, mae]
    is_error_higher = mse > best_mse or mae > best_mae
    if i != 0 and is_error_higher: continue
    best_mse, best_mae, best_model = mse, mae, models[i][0]
  return {
    'best_model': best_model,
    'best_mse': best_mse,
    'best_mae': best_mae,
    'results': results
  }

The function below returns the result of function above but also displays the results

In [10]:
def view_results(title, models, X, y):
  data = compare_models(models, X, y)
  print('\033[94m' + title + '\033[0m')
  print('\033[92m' + data['best_model'] + '\033[0m' + f" is the best model for the {title}.")
  print("MSE: ", '\033[92m' + str(data['best_mse']) + '\033[0m')
  print("MAE: ", '\033[92m' + str(data['best_mae']) + '\033[0m')
  display(data['results'])
  return data

Getting the errors and displaying them for the test data set

In [11]:
test_data_result = view_results('Testing Data', models, X_test, y_test)

Testing Data
Linear Regression is the best model for the Testing Data.
MSE:  0.049965083884116385
MAE:  0.0981108639850805


,mse,mae
Linear Regression,0.049965,0.098111
Random Forest,0.051651,0.092270
Gradient Boosting,0.049991,0.088170
Decision Tree,0.098500,0.098500
K Neighbors,0.056751,0.089106
Ridge,0.049965,0.098099
Lasso,0.054241,0.096983
Elastic Net,0.051967,0.094233
TheilSen,0.058044,0.058092


Likewise as above but with the Training data set

In [12]:
train_data_result = view_results('Training Data', models, X_train, y_train)

Training Data
Decision Tree is the best model for the Training Data.
MSE:  0.0
MAE:  0.0


,mse,mae
Linear Regression,0.039310,0.087239
Random Forest,0.006269,0.031834
Gradient Boosting,0.030024,0.067792
Decision Tree,0.000000,0.000000
K Neighbors,0.031770,0.063573
Ridge,0.039310,0.087230
Lasso,0.042259,0.084962
Elastic Net,0.040673,0.083005
TheilSen,0.044721,0.044768


Calculating the differences to check for underfitting and overfitting

In [13]:
diff_df = pd.DataFrame(columns=['mse difference', 'mae difference'])
diff_df['mse difference'] = train_data_result['results']['mse'] - test_data_result['results']['mse']
diff_df['mae difference'] = train_data_result['results']['mae'] - test_data_result['results']['mae']
display(diff_df)

,mse difference,mae difference
Linear Regression,-0.010655,-0.010872
Random Forest,-0.045382,-0.060436
Gradient Boosting,-0.019968,-0.020378
Decision Tree,-0.098500,-0.098500
K Neighbors,-0.024982,-0.025534
Ridge,-0.010655,-0.010870
Lasso,-0.011982,-0.012021
Elastic Net,-0.011294,-0.011228
TheilSen,-0.013323,-0.013323


Since there seems to be a small difference between testing and training data for the Gradient Boosting Regressor, I will choose this model for grid search

### Doing Grid Search on the Gradient Boosting Regressor to find best params

In [14]:
params = {
  'learning_rate': [0.1, 0.05, 0.02, 0.01],
  'n_estimators': [100, 300, 500, 1000],
  'max_depth': [4, 6],
  'subsample': [0.5, 0.75, 1.0],
  'min_samples_leaf': [3, 5, 9, 17],
  'max_features': [1.0, 0.3, 0.1],
}

In [15]:
GBR = GradientBoostingRegressor()

In [16]:
GSCV = GridSearchCV(GBR, params, n_jobs=3, verbose=1)
GSCV.fit(X_train, y_train)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


GridSearchCV(estimator=GradientBoostingRegressor(), n_jobs=3,
             param_grid={'learning_rate': [0.1, 0.05, 0.02, 0.01],
                         'max_depth': [4, 6], 'max_features': [1.0, 0.3, 0.1],
                         'min_samples_leaf': [3, 5, 9, 17],
                         'n_estimators': [100, 300, 500, 1000],
                         'subsample': [0.5, 0.75, 1.0]},
             verbose=1)

In [21]:
GSCV.best_params_

{'learning_rate': 0.02,
 'max_depth': 4,
 'max_features': 0.3,
 'min_samples_leaf': 17,
 'n_estimators': 100,
 'subsample': 0.5}

In [22]:
best_gbr = GSCV.best_estimator_

In [23]:
best_gbr.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.02, max_depth=4, max_features=0.3,
                          min_samples_leaf=17, subsample=0.5)

### Saving the model to a file

In [26]:
dump(best_gbr, 'models/gbr.joblib')

['models/gbr.joblib']